In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re
import os
import sys
import javalang
from xml.dom.minidom import parse
import xml.dom.minidom

In [ ]:
def variable_query(root, name, target_list):
    children = None
    if root is None:
        #print(root)
        return 1
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    for child in children:
        if isinstance(child, (javalang.tree.VariableDeclarator)) and child.name == name:
            target_list.append(child)
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            variable_query(child,name,target_list)

            
def permission_check(permission_list,file_path):
    try:
        DOMTree = xml.dom.minidom.parse(file_path)
    except:
        return False
    collection = DOMTree.documentElement
    permission_tags = collection.getElementsByTagName('uses-permission')
    for tag in permission_tags:
        name = tag.getAttribute("android:name")
        if name in permission_list:
            return False
    return True
def factors_check(name_list,text):
    for name in name_list:
        if name in text:
            return True
    return False

In [ ]:
def walk_tree_media_Hed(root,violation_list,media_dict):
    children = None
    if root is None:
        #print(root)
        return 
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    
    for child in children:
        
        if isinstance(child, (javalang.tree.Import)):
            if 'com.google.android.exoplayer' in child.path:
                violation_list.append('violation')
            
        if isinstance(child, (javalang.tree.FieldDeclaration,javalang.tree.LocalVariableDeclaration)):
            if 'MediaPlayer' in child.type.name:
                for declarator in child.declarators:
                    if isinstance(declarator, (javalang.tree.VariableDeclarator)):
                        media_dict[declarator.name] = list()
                        
        if isinstance(child, javalang.tree.This):
            for member in child.selectors:
                if isinstance(member, (javalang.tree.MethodInvocation)):
                    #print(member.member)
                    if isinstance(child.selectors[0], javalang.tree.MemberReference):
                        #print(child.selectors[0].member)
                        if child.selectors[0].member in media_dict.keys() and \
                        (member.member == 'start' or member.member == 'pause'or member.member == 'stop'or member.member == 'release'):
                            media_dict[child.selectors[0].member].append(member.member)
        
        if isinstance(child, (javalang.tree.MethodInvocation)):
            if child.qualifier in media_dict.keys() \
            and (child.member == 'start' or child.member == 'pause' or child.member == 'stop'or child.member == 'release'):
                media_dict[child.qualifier].append(child.member)
                
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_media_Hed(child,violation_list,media_dict)

            
def walk_tree_media_Uni(root,violation_list,media_dict):
    children = None
    if root is None:
        #print(root)
        return 
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    
    for child in children:
            
        if isinstance(child, (javalang.tree.FieldDeclaration,javalang.tree.LocalVariableDeclaration)):
            if 'MediaPlayer' in child.type.name:
                for declarator in child.declarators:
                    if isinstance(declarator, (javalang.tree.VariableDeclarator)):
                        media_dict[declarator.name] = list()
                        
        if isinstance(child, javalang.tree.This):
            for member in child.selectors:
                if isinstance(member, (javalang.tree.MethodInvocation)):
                    #print(member.member)
                    if isinstance(child.selectors[0], javalang.tree.MemberReference):
                        #print(child.selectors[0].member)
                        if child.selectors[0].member in media_dict.keys() and \
                        (member.member == 'start' or member.member == 'pause'or member.member == 'stop'or member.member == 'release'):
                            media_dict[child.selectors[0].member].append(member.member)
        
        if isinstance(child, (javalang.tree.MethodInvocation)):
            if child.qualifier in media_dict.keys() \
            and (child.member == 'start' or child.member == 'pause' or child.member == 'stop'or child.member == 'release'):
                media_dict[child.qualifier].append(child.member)
                
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_media_Uni(child,violation_list,media_dict)


In [ ]:
def walk_tree_media_SD(root,violation_list):
    children = None
    if root is None:
        #print(root)
        return 
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    
    for child in children:
        
        if isinstance(child, (javalang.tree.Import)):
            if 'com.google.android.exoplayer' in child.path:
                violation_list.append('violation')
                
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_media_SD(child,violation_list)


In [ ]:
def media_SD_detector(file_path,text,treenode):
    if text == False and treenode == False :
        return 0
    factor_list = ['com.google.android.exoplayer']
    if not factors_check(factor_list, text):
        return 0
    
    violation_list = list()
    walk_tree_media_SD(treenode,violation_list)
    if 'violation' in violation_list:
        print('media_Uni violation')
        return 1
    return 0

def media_Uni_detector(file_path,text,treenode):
    if text == False and treenode == False :
        return 0
    factor_list = ['MediaPlayer', 'start','pause', 'stop']
    if not factors_check(factor_list, text):
        return 0
    return 0
    
    violation_list = list()
    name_list = list()
    media_dict = dict()
    walk_tree_media_Uni(treenode,violation_list,media_dict)
    if 'violation' in violation_list:
        print('media_hedonism_SD violation')
        return 1
    for key,item in media_dict.items():
        if 'start' in item and (not 'pause' in item or not 'stop' in item):
            print('media_hedonism_SD violation')
            return 1
    return 0

In [ ]:
def media_hedonism_detector(file_path,text,treenode):
    if text == False and treenode == False :
        return 0
    factor_list = ['com.google.android.exoplayer', 'MediaPlayer', 'start','pause', 'stop']
    if not factors_check(factor_list, text):
        return 0
    return 0
    
    violation_list = list()
    name_list = list()
    media_dict = dict()
    walk_tree_media_Hed(treenode,violation_list,media_dict)
    if 'violation' in violation_list:
        print('media_hedonism violation')
        return 1
    for key,item in media_dict.items():
        if 'start' in item and (not 'pause' in item or not 'stop' in item):
            print('media_hedonism violation')
            return 1
    return 0

In [ ]:
i=0

pridir_code = 'D:/jupyterCode/summer_object/apks/decomplied/'
for code_folder in os.listdir(pridir_code)[:]:
    #with open('text.txt','r') as file:
    #    code_folder = file.read()
    if code_folder.endswith(".zip"):
        continue
        
    row = {'apk_name':code_folder,'mtp_SD':0,'hardware_security':0,'nfc_SD':0,'animation_hedonism':0, \
           'media_SD':0,'media_hedonism':0,'media_Uni':0,'nfc_security':0,\
       'telephony_security':0,'telephony_conformity':0}
    full_code_folder = pridir_code+code_folder
    for root, dirs, files in os.walk(full_code_folder):
        for file in files:

            try:
                if file.endswith(".java"):
                    file_path = os.path.join(root, file)

                    try:
                        with open(file_path, "r") as file:
                            text = file.read()
                        treenode = javalang.parse.parse(text)
                    except:
                        text = False
                        treenode = False
                        continue
                    #medias
                    if not row['media_hedonism']:
                        row['media_hedonism'] = media_hedonism_detector(file_path,text,treenode)
                        row['media_SD'] = row['media_hedonism']
                    if not row['media_SD']:
                        row['media_SD'] = media_SD_detector(file_path,text,treenode)
                        row['media_hedonism'] = row['media_SD']

                    if not row['media_Uni']:
                        row['media_Uni'] = media_Uni_detector(file_path,text,treenode)
                        if row['media_Uni']:
                            row['media_hedonism'] = row['media_Uni']


            except:
                print('error')
    i+=1